In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("predictions/zeolite_openweight_results_nonull.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna(subset=["model"])
df.head()

,doi,model,task,modality,Si,Ge,Al,OH,H2O,HF,B,SDA_Type,SDA,Temp,Time,Extracted
0,10.1007/s11244-013-0170-7,llama3.3-70,zeolite,xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,2.0,NaN
1,10.1007/s11244-013-0170-7,llama3.3-70,zeolite,xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,2.0,NaN
2,10.1007/s11244-013-0170-7,llama3.3-70,zeolite,xml,1,1,0.024,NaN,NaN,NaN,NaN,NaN,42,400.0,NaN,NaN
3,10.1007/s11244-013-0170-7,llama3.3-70,zeolite,xml,1,1,0.01,NaN,NaN,NaN,NaN,NaN,>100,400.0,NaN,NaN
4,10.1007/s11244-013-0170-7,llama3.3-70,zeolite,xml,1,2,1/42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df["modality"].fillna("missing").value_counts()

modality
xml       3006
images    2868
Name: count, dtype: int64

In [10]:
df["model"].value_counts()

model
gpt4o           2770
llama3.3-70     1384
molmo           1046
qwen2.5VL-72     674
Name: count, dtype: int64

In [11]:
df.groupby(["model", "modality"]).count()

doi  task    Si    Ge    Al    OH   H2O   HF    B  \
model        modality                                                       
gpt4o        images    1148  1148   915   907   283   267   555  300   68   
             xml       1622  1622  1223  1254   555   494   674  432   72   
llama3.3-70  xml       1384  1384  1161  1060   568   424   551  273   55   
molmo        images    1046  1046  1046  1046  1034  1022  1015  893  707   
qwen2.5VL-72 images     674   674   541   494   130   183   294  161   31   

                       SDA_Type  SDA  Temp  Time  Extracted  
model        modality                                        
gpt4o        images         621  393   627   576        417  
             xml            732  302   751   593        969  
llama3.3-70  xml            682  439   532   398        241  
molmo        images         810  912   117     6        869  
qwen2.5VL-72 images         432  336   358   309        462

In [12]:
for group_name, group_idx  in df.groupby(["task", "model", "modality"]).groups.items():
    if pd.isna(group_name[0]):
        continue
    group_df = df.loc[group_idx]
    task, model, raw_modality = group_name
    if raw_modality == "xml":
        modality = "xml"
    else:
        modality = "pdf"
    group_df.rename(columns={"SDA_Type": "SDA Type"}).to_csv(f"predictions/{task}_{model}_{modality}.csv", index=False)
